##DeepLearning Project in 2020

In [2]:
pip install konlpy

     |████████████████████████████████| 19.4MB 161kB/s 
     |████████████████████████████████| 92kB 12.1MB/s 
     |████████████████████████████████| 3.6MB 51.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [68]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###데이터 로드

In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7fdf3ad6f898>)

In [0]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [6]:
print('훈련용 리뷰 개수 :',len(train_data)) # 훈련용 리뷰 개수 출력

훈련용 리뷰 개수 : 150000


In [7]:
train_data[:5]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [8]:
print('테스트 리뷰 개수:', len(test_data))

테스트 리뷰 개수: 50000


In [9]:
test_data[:5]

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


###데이터 전처리

In [10]:
train_data['document'].nunique() # document에서 중복 데이터가 몇 개인지 찾기

146182

In [11]:
train_data['label'].nunique()

2

In [0]:
train_data = train_data.drop_duplicates('document', keep='first')
# 중복되는 데이터 중 처음 것을 남겨두고 나머지 제거

In [0]:
# train_data.drop_duplicates(subset=['document'], inplace=True)

In [14]:
print('the number of document', len(train_data))

the number of document 146183


In [15]:
print(train_data)

              id                                           document  label
0        9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1        3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2       10265843                                  너무재밓었다그래서보는것을추천한다      0
3        9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4        6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
...          ...                                                ...    ...
149995   6222902                                인간이 문제지.. 소는 뭔죄인가..      0
149996   8549745                                      평점이 너무 낮아서...      1
149997   9311800                    이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?      0
149998   2376369                        청춘 영화의 최고봉.방황과 우울했던 날들의 자화상      1
149999   9619869                           한국 영화 최초로 수간하는 내용이 담긴 영화      0

[146183 rows x 3 columns]


"No matching overloads found for kr.lucypark.okt.OktInterface.tokenize" 이러한 에러로 인해 null 값을 제거해줌

In [16]:
print(train_data.isnull().sum()) #null값 있는지 알아보기

id          0
document    1
label       0
dtype: int64


In [0]:
train_data = train_data.dropna(axis=0) #null값 제거

In [18]:
print(train_data.isnull().sum())

id          0
document    0
label       0
dtype: int64


In [19]:
print(test_data.isnull().sum())

id          0
document    3
label       0
dtype: int64


In [0]:
test_data = test_data.dropna(axis=0)

In [21]:
print(test_data.isnull().sum())

id          0
document    0
label       0
dtype: int64


###불용어 제거 및 토큰화

전에 미리 불용어 데이터를 구글 드라이브에 넣어둠

In [0]:
stop_words_list = []
stopwords = open('./drive/My Drive/Colab Notebooks/data/stopwords.txt', 'r', encoding='utf-8')

for line in stopwords.readlines():
  stop_words_list.append(line.rstrip())
  # print(stop_words_list)
stopwords.close()

In [26]:
print(stop_words_list)

['이', '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하', '때문', '그것', '두', '말하', '알', '그러나', '받', '못하', '일', '그런', '또', '문제', '더', '사회', '많', '그리고', '좋', '크', '따르', '중', '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집', '살', '모르', '적', '월', '데', '자신', '안', '어떤', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', '다른', '어떻', '여자', '개', '전', '들', '사실', '이렇', '점', '싶', '말', '정도', '좀', '원', '잘', '통하', '소리', '놓']


In [0]:
okt = Okt()
x_train = []
for sentence in train_data['document']:
  temp = []
  temp = okt.morphs(sentence, stem=True)
  temp = [word for word in temp if not word in stop_words_list]
  # print(temp)
  x_train.append(temp)

In [31]:
print(x_train[:3])

[['아', '더빙', '..', '진짜', '짜증나다', '목소리'], ['흠', '...', '포스터', '보고', '초딩', '영화', '줄', '....', '오버', '연기', '조차', '가볍다', '않다'], ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다']]


In [0]:
okt = Okt()
x_test = []
for sentence in test_data['document']:
    temp = []
    temp = okt.morphs(sentence, stem=True)
    temp = [word for word in temp if not word in stop_words_list]
    x_test.append(temp)
    # print(temp)

In [37]:
print(x_test[:3])

[['굳다', 'ㅋ'], ['GDNTOPCLASSINTHECLUB'], ['뭐', '야', '평점', '은', '....', '나쁘다', '않다', '10', '짜다', '리', '는', '더욱', '아니다']]


In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

In [53]:
print(tokenizer.word_index)

{'.': 1, '영화': 2, '보다': 3, '하다': 4, '의': 5, '..': 6, '에': 7, '을': 8, '...': 9, '도': 10, ',': 11, '는': 12, '를': 13, '은': 14, '없다': 15, '이다': 16, '있다': 17, '좋다': 18, '너무': 19, '?': 20, '다': 21, '정말': 22, '되다': 23, '재밌다': 24, '만': 25, '같다': 26, '진짜': 27, '!': 28, '으로': 29, '로': 30, '아니다': 31, '않다': 32, '에서': 33, '만들다': 34, '연기': 35, '과': 36, '나오다': 37, '평점': 38, '~': 39, '최고': 40, '인': 41, '스토리': 42, '못': 43, '....': 44, '왜': 45, '드라마': 46, '게': 47, '감동': 48, '1': 49, '보고': 50, '이렇다': 51, '고': 52, '아깝다': 53, '아': 54, '배우': 55, 'ㅋㅋ': 56, '와': 57, '거': 58, '감독': 59, '그냥': 60, '재미있다': 61, '요': 62, '내용': 63, '재미': 64, '뭐': 65, '까지': 66, '!!': 67, '주다': 68, '자다': 69, '하고': 70, '지루하다': 71, '재미없다': 72, '쓰레기': 73, '가다': 74, '네': 75, '모르다': 76, '들다': 77, '그렇다': 78, '싶다': 79, '작품': 80, '10': 81, '사랑': 82, '알다': 83, '마지막': 84, '볼': 85, '이건': 86, '오다': 87, 'ㅋ': 88, '저': 89, '완전': 90, '2': 91, 'ㅋㅋㅋ': 92, '많다': 93, '처음': 94, '장면': 95, '액션': 96, '주인공': 97, '차다': 98, '이렇게': 99, '걸': 100, '안되다': 101, '나다'

In [54]:
delete = 3
total_cnt = len(tokenizer.word_index)
rare_cnt = 0
total_freq = 0
rare_freq = 0

for key, value in tokenizer.word_counts.items():
  total_freq = total_freq + value

  if(value < delete):
    rare_cnt = rare_cnt + 1
    rare_freq = rare_freq + value

print('단어 집합 크기 : ', total_cnt)
print('등장 빈도가 %s 번 이하인 희귀 단어의 수 : %s' %(delete - 1, rare_cnt))
print('단어 집합에서 희귀 단어의 비율 : ', (rare_cnt / total_cnt) * 100)
print('전체 등장 빈도에서 희귀 단어 등장 빈도 비율 : ', (rare_freq / total_freq) * 100)

단어 집합 크기 :  49532
등장 빈도가 2 번 이하인 희귀 단어의 수 : 28785
단어 집합에서 희귀 단어의 비율 :  58.11394653961076
전체 등장 빈도에서 희귀 단어 등장 빈도 비율 :  1.8505594677950759


In [55]:
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합 크기 : ', vocab_size)

단어 집합 크기 :  20748


In [0]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [57]:
print(x_train[:3])

[[54, 453, 6, 27, 254, 661], [918, 9, 454, 50, 608, 2, 209, 44, 1482, 35, 967, 679, 32], [380, 2525, 5252, 6568, 3, 215, 21]]


In [0]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

In [0]:
train_empty_sample = [index for index, sentence in enumerate(x_train) if len(sentence) < 1]
test_empty_sample = [index for index, sentence in enumerate(x_test) if len(sentence) < 1]

In [61]:
x_train = np.delete(x_train, train_empty_sample, axis=0)
y_train = np.delete(y_train, train_empty_sample, axis=0)
print(len(x_train))
print(len(y_train))

145556
145869


In [63]:
x_test = np.delete(x_test, test_empty_sample, axis=0)
y_test = np.delete(y_test, test_empty_sample, axis=0)
print(len(x_test))
print(len(y_test))

49758
49758


In [0]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='relu'))



In [80]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=15, batch_size=60, validation_split=0.2)

ValueError: ignored